<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-16 09:23:47
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.38 C
-------------------
Today PnL: 39.77 K (0.29%)
Current PnL: -12.73 L (-8.87%)
CY Booked + Current PnL: -5.36 L (-3.73%)
-------------------
Total profit:  4.70 L
Total loss:  -17.42 L
-------------------
Total Booked + Current PnL: 21.62 L (18.56%)
Total Booked PnL: 34.35 L (29.47%)
Curr Year Booked PnL: 7.37 L (5.35%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.03 L (51.55%)
Deployed:  1.17 C
Current:  1.38 C
CAGR/XIRR %: 11.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-230.45,H-SC,95.83,94088.0,-1228.0,1270.0,-0.09,-1.29,1.35,0.04,162.0,-0.97,0.68,85.58,OX40N,NTT,MISC
86,UJJIVANSFB,52.77,53.0,55.19,M-SC,56.25,134649.0,-7830.0,8456.0,0.42,-5.50,6.28,0.44,251.0,-0.93,0.98,58.92,OX40N,NTT,BANKS
70,SBIN,760.30,863.0,-10.56,M-LC,57.29,216796.0,15316.0,11902.0,0.20,7.60,5.49,13.51,61.0,1.29,1.57,21.16,XY25,NTT,BANKS
80,TATAELXSI,7332.28,7332.0,-14.62,X-MC,40.62,89705.0,-12947.0,12944.0,0.43,-12.61,14.43,-0.00,54.0,-1.00,0.65,36.36,OX40N,NTT,IT
4,APOLLOHOSP,6904.43,8285.0,-18.95,H-LC,58.33,110340.0,6774.0,13936.0,-0.06,6.54,12.63,20.00,14.0,0.49,0.80,21.53,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,MASFIN,326.60,399.50,-14.18,H-SC,88.54,97425.0,-555.0,22427.0,2.92,-0.57,23.02,22.32,146.0,-0.02,0.71,41.30,XR,ATH,FINANCE
14,BERGEPAINT,561.33,680.00,-19.64,H-MC,23.96,227549.0,210.0,47854.0,0.12,0.09,21.03,21.14,109.0,0.00,1.65,27.40,XY24,NTT,PAINTS
12,BANKINDIA,116.91,190.00,-32.52,H-MC,31.25,178716.0,-1092.0,113503.0,0.35,-0.61,63.51,62.52,93.0,-0.01,1.30,33.27,XR,NTT,BANKS
61,PGHH,13388.00,18172.17,-20.74,H-MC,33.33,199905.0,-915.0,72685.0,0.99,-0.46,36.36,35.73,92.0,-0.01,1.45,4.20,X40,ATH,FMCG
59,NATIONALUM,189.63,247.44,-50.15,H-MC,44.79,100504.0,0.0,30644.0,-0.45,0.00,30.49,30.49,83.0,0.00,0.73,32.40,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,93.75,75782.0,-37767.0,77821.0,8.64,-33.26,102.69,35.27,268.0,-0.49,0.55,85.48,XR,NTT,HOTELS
35,HATHWAY,19.74,31.40,1092.03,L-SC,94.79,68360.0,-10600.0,57238.0,8.58,-13.42,83.73,59.07,269.0,-0.19,0.50,37.16,XR,NTT,ENTERTAINMENT
28,DIGITIDE,188.38,246.27,NaN,NaN,60.42,53933.0,12678.0,0.0,5.56,30.73,0.00,30.73,196.0,inf,0.39,26.98,XY24,ATH,IT
29,DIXON,14367.00,18940.15,-28.95,H-MC,96.88,130864.0,15928.0,20663.0,3.38,13.86,15.79,31.83,84.0,0.77,0.95,49.99,X40N,ATH,IT
16,BLUSPRING,226.45,86.36,NaN,NaN,97.92,18332.0,-31261.0,581.0,3.25,-63.03,3.17,-61.86,201.0,-53.81,0.13,22.94,XY24,ATH,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ICICIPRULI,600.83,790.00,-9.47,H-MC,45.83,146290.0,10502.0,32257.0,-3.33,7.73,22.05,31.48,113.0,0.33,1.06,20.79,X40,ATH,INSURANCE
37,HCLTECH,1647.60,1959.14,7.58,H-LC,3.12,95550.0,-4954.0,23954.0,-3.30,-4.93,25.07,18.91,12.0,-0.21,0.69,15.31,X40,ATH,IT
67,SAIL,130.64,228.00,91.93,M-MC,61.46,229904.0,4942.0,162703.0,-1.02,2.20,70.77,74.53,194.0,0.03,1.67,33.48,XY24,BTT,STEEL
94,WHIRLPOOL,1167.49,2270.00,-38.81,M-SC,62.50,201566.0,29945.0,132127.0,-0.91,17.45,65.55,94.43,205.0,0.23,1.46,48.77,XR,NTT,DURABLES
22,CIPLA,1495.00,1795.00,-23.18,H-LC,15.62,99026.0,-1139.0,21241.0,-0.87,-1.14,21.45,20.07,10.0,-0.05,0.72,7.66,X40N,BTT,PHARMA


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-230.45,H-SC,95.83,94088.0,-1228.0,1270.0,-0.09,-1.29,1.35,0.04,162.0,-0.97,0.68,85.58,OX40N,NTT,MISC
86,UJJIVANSFB,52.77,53.0,55.19,M-SC,56.25,134649.0,-7830.0,8456.0,0.42,-5.50,6.28,0.44,251.0,-0.93,0.98,58.92,OX40N,NTT,BANKS
78,SYMPHONY,1306.42,1306.0,-27.09,M-SC,65.62,155497.0,-15644.0,15596.0,1.43,-9.14,10.03,-0.03,195.0,-1.00,1.13,11.76,OX40N,NTT,DURABLES
80,TATAELXSI,7332.28,7332.0,-14.62,X-MC,40.62,89705.0,-12947.0,12944.0,0.43,-12.61,14.43,-0.00,54.0,-1.00,0.65,36.36,OX40N,NTT,IT
46,INDIGOPNTS,1407.73,1408.0,128.24,H-SC,91.67,149494.0,-25065.0,25100.0,0.42,-14.36,16.79,0.02,129.0,-1.00,1.09,29.63,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,89.58,76275.0,41826.0,3143.0,-0.09,121.42,4.12,130.54,1.0,13.31,0.55,16.64,AR,ATH,ELECTRICAL
41,HINDZINC,514.80,730.22,16.43,H-LC,1.04,87460.0,-15500.0,58581.0,-0.03,-15.05,66.98,41.85,2.0,-0.26,0.64,15.11,X5K,ATH,METALS
91,VBL,492.64,672.28,-16.92,H-LC,28.12,252161.0,-12879.0,109539.0,-0.23,-4.86,43.44,36.46,3.0,-0.12,1.83,7.69,X40N,ATH,FMCG
82,TCS,3794.03,4998.00,-22.56,H-LC,7.29,252728.0,-43206.0,137105.0,-0.04,-14.60,54.25,31.73,4.0,-0.32,1.84,1.50,X40,BTT,IT
48,INFY,1461.46,2275.00,-11.34,H-LC,18.75,284825.0,23224.0,122389.0,0.35,8.88,42.97,55.67,6.0,0.19,2.07,15.52,X40,BTT,IT


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,246.27,NaN,NaN,60.42,53933.0,12678.0,0.0,5.56,30.73,0.00,30.73,196.0,inf,0.39,26.98,XY24,ATH,IT
58,MEDANTA,1064.65,1486.00,21.31,H-SC,86.46,258620.0,53143.0,28190.0,1.04,25.86,10.90,39.58,153.0,1.89,1.88,38.59,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,89.58,76275.0,41826.0,3143.0,-0.09,121.42,4.12,130.54,1.0,13.31,0.55,16.64,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,91.93,M-MC,61.46,229904.0,4942.0,162703.0,-1.02,2.20,70.77,74.53,194.0,0.03,1.67,33.48,XY24,BTT,STEEL
53,KPIGREEN,497.21,731.97,33.09,H-SC,83.33,136332.0,11035.0,48125.0,0.18,8.81,35.30,47.22,141.0,0.23,0.99,70.56,MH,ATH,POWER
58,MEDANTA,1064.65,1486.00,21.31,H-SC,86.46,258620.0,53143.0,28190.0,1.04,25.86,10.90,39.58,153.0,1.89,1.88,38.59,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,6.09,H-SC,63.54,119218.0,17214.0,27134.0,0.22,16.88,22.76,43.48,122.0,0.63,0.87,36.49,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,0.09,H-SC,92.71,187810.0,23140.0,45938.0,-0.22,14.05,24.46,41.95,119.0,0.50,1.36,22.57,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,89.58,76275.0,41826.0,3143.0,-0.09,121.42,4.12,130.54,1.0,13.31,0.55,16.64,AR,ATH,ELECTRICAL
53,KPIGREEN,497.21,731.97,33.09,H-SC,83.33,136332.0,11035.0,48125.0,0.18,8.81,35.30,47.22,141.0,0.23,0.99,70.56,MH,ATH,POWER
94,WHIRLPOOL,1167.49,2270.00,-38.81,M-SC,62.50,201566.0,29945.0,132127.0,-0.91,17.45,65.55,94.43,205.0,0.23,1.46,48.77,XR,NTT,DURABLES
45,INDIAMART,2327.09,4850.92,-44.71,H-SC,84.38,143948.0,20612.0,113158.0,0.10,16.71,78.61,108.45,118.0,0.18,1.05,42.80,AR,ATH,MISC
95,WIPRO,243.46,420.00,-5.58,M-LC,26.04,161386.0,10441.0,99010.0,1.13,6.92,61.35,72.51,57.0,0.11,1.17,11.17,XR,NTT,IT


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.36,NaN,NaN,97.92,18332.0,-31261.0,581.0,3.25,-63.03,3.17,-61.86,201.0,-53.81,0.13,22.94,XY24,ATH,MISC
28,DIGITIDE,188.38,246.27,NaN,NaN,60.42,53933.0,12678.0,0.0,5.56,30.73,0.00,30.73,196.0,inf,0.39,26.98,XY24,ATH,IT
74,SIS,477.00,477.00,2298.45,M-SC,52.08,64532.0,-16558.0,16559.0,1.17,-20.42,25.66,0.00,237.0,-1.00,0.47,30.78,OX40N,NTT,MISC
35,HATHWAY,19.74,31.40,1092.03,L-SC,94.79,68360.0,-10600.0,57238.0,8.58,-13.42,83.73,59.07,269.0,-0.19,0.50,37.16,XR,NTT,ENTERTAINMENT
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,93.75,75782.0,-37767.0,77821.0,8.64,-33.26,102.69,35.27,268.0,-0.49,0.55,85.48,XR,NTT,HOTELS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,RELAXO,902.64,1176.00,-29.89,H-SC,100.00,63398.0,-52140.0,87128.0,0.13,-45.13,137.43,30.28,131.0,-0.60,0.46,23.86,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,7.09,H-SC,98.96,99631.0,-1332.0,95865.0,0.47,-1.32,96.22,93.63,150.0,-0.01,0.72,48.40,SR,ATH,CHEMICALS
16,BLUSPRING,226.45,86.36,NaN,NaN,97.92,18332.0,-31261.0,581.0,3.25,-63.03,3.17,-61.86,201.0,-53.81,0.13,22.94,XY24,ATH,MISC
29,DIXON,14367.00,18940.15,-28.95,H-MC,96.88,130864.0,15928.0,20663.0,3.38,13.86,15.79,31.83,84.0,0.77,0.95,49.99,X40N,ATH,IT
92,VIPIND,488.80,489.00,-230.45,H-SC,95.83,94088.0,-1228.0,1270.0,-0.09,-1.29,1.35,0.04,162.0,-0.97,0.68,85.58,OX40N,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.86
1,25,41.56
2,50,70.38


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.94
LC    34.54
MC    21.43
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.76
X40N     13.27
X40      12.58
XY25     12.09
XR       10.09
OX40N     8.32
AR        8.16
MH        1.72
X5K       1.41
X200      1.38
SR        1.20
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.76
H-SC    25.83
H-MC    17.69
M-SC    15.59
M-LC     5.74
M-MC     2.72
L-SC     1.52
L-LC     1.04
X-MC     0.65
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.18,-0.07,33.59
IT,11.89,-10.88,65.44
FINANCE,8.46,-12.07,52.73
BANKS,7.76,-6.76,50.33
MISC,6.01,-16.22,70.26
PAINTS,5.55,-14.31,36.84
ELECTRICAL,5.32,-0.66,48.36
HEALTHCARE,4.83,3.16,25.12
AUTO,4.28,-14.53,64.89


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2592776.0,25
XR,1069962.0,13
AR,1025700.0,10
X40,722894.0,10
X40N,551453.0,12
XY25,455059.0,8
OX40N,285411.0,11
SR,177657.0,2
MH,78769.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2291288.0,26
M-SC,1648103.0,20
H-LC,1246314.0,21
H-MC,1039397.0,14
M-MC,303223.0,2
M-LC,268968.0,4
L-SC,190982.0,3
L-MC,52230.0,1
L-LC,46208.0,1


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,907238.0,7
M-SC,XY24,786095.0,7
H-SC,AR,509582.0,3
H-LC,X40,464283.0,6
H-MC,XY24,390550.0,4
M-SC,XR,335550.0,4
H-SC,XR,334610.0,4
M-MC,XY24,303223.0,2
H-LC,X40N,286266.0,6


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
